In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from tqdm import tqdm
import numpy as np

In [70]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        # Define the layers
        self.fc1 = nn.Linear(26, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 3)
        self.fc4 = nn.Linear(3, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 26) 

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        
        return x
loss_function = nn.MSELoss()
model = NeuralNetwork()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True, min_lr=1e-6)


In [56]:
#encode the alphabet into one-hot tensors
def encode(letter):
    vector = torch.zeros(1, 26)
    vector[0][ord(letter) - 97] = 1
    return vector

data = []
labels = []
for i in range(26):
    data.append(encode(chr(i + 97)))
    labels.append(encode(chr(i + 97)))

In [71]:
epochs = 2000
for epoch in range(epochs+1):
    epoch_loss = 0
    for i in range(len(data)):
        model.zero_grad()
        output = model(data[i])
        loss = loss_function(output, labels[i])
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    scheduler.step(epoch_loss/len(data))
    if epoch % 100 == 0:
        print('Epoch: ', epoch, 'Loss: ', loss.item())

Epoch:  0 Loss:  0.03540126979351044
Epoch:  100 Loss:  0.03352459892630577
Epoch:  200 Loss:  0.02766551449894905
Epoch:  300 Loss:  0.02271033264696598
Epoch:  400 Loss:  0.02013450860977173
Epoch:  500 Loss:  0.015553519129753113
Epoch:  600 Loss:  0.010116183198988438
Epoch:  700 Loss:  0.006018269341439009
Epoch:  800 Loss:  0.0035006776452064514
Epoch:  900 Loss:  0.0019083862425759435
Epoch:  1000 Loss:  0.0008522673160769045
Epoch:  1100 Loss:  0.0003212998271919787
Epoch:  1200 Loss:  0.00013152151950635016
Epoch:  1300 Loss:  6.1125545471441e-05
Epoch 01327: reducing learning rate of group 0 to 5.0000e-05.
Epoch:  1400 Loss:  2.282373679918237e-05
Epoch:  1500 Loss:  2.5286060918006115e-05
Epoch:  1600 Loss:  2.1109543013153598e-05
Epoch 01691: reducing learning rate of group 0 to 2.5000e-05.
Epoch:  1700 Loss:  2.1006881070206873e-05
Epoch:  1800 Loss:  2.481002957210876e-05
Epoch:  1900 Loss:  2.298479012097232e-05
Epoch:  2000 Loss:  2.526309253880754e-05


In [72]:
#efficiency test, ie using an argmax function to determine the letter, is it wrong?
num_wrong = 0
for i in tqdm(range(int(26))):
    let = chr(i + 97)
    letter = encode(let)
    output = model(letter)
    output = output.detach().numpy()
    output = np.argmax(output)
    if output != ord(let) - 97:
        num_wrong += 1
print(num_wrong)

100%|██████████| 26/26 [00:00<00:00, 988.48it/s]

0
